
# Install and import libraries

In [ ]:
pip install diffrax

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.7/141.7 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.9/167.9 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.3/65.3 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.4/89.4 kB 10.4 MB/s eta 0:00:00


In [ ]:
pip install h5py typing-extensions wheel

In [ ]:
pip install scipy cvxpy bagpy pyquaternion

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.6/90.6 kB 449.6 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.6/133.6 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.3/288.3 kB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 60.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.4/100.4 kB 7.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 9.1 MB/s eta 0:00:00
  Created wheel for gnupg: filename=gnupg-2.3.1-py3-none-any.whl size=94617 sha256=4c94199924af8cb6965ba1d06c7e3ea9b1e3964ec9f1ff2f5ac2324d17a5ff87
  St

In [ ]:
# !sudo apt install cm-super dvipng texlive-latex-extra texlive-latex-recommended

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import time

import diffrax
import equinox as eqx  # https://github.com/patrick-kidger/equinox
import jax
import jax.nn as jnn
import jax.numpy as jnp
import jax.random as jrandom
import matplotlib.pyplot as plt
import optax  # https://github.com/deepmind/optax
from sklearn.preprocessing import MinMaxScaler
from scipy import interpolate

# import equinox.experimental as eqxe
import jax.tree_util as jtu
import functools as ft

import cvxpy as cp

import plotly.graph_objects as go

import numpy as np
from matplotlib.lines import Line2D
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from numpy import savetxt

# Read and plot the demo fron a .npy file

In [ ]:
import numpy as np
import pandas as pd
from matplotlib.lines import Line2D
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import plotly.graph_objects as go
# save numpy array as csv file
from numpy import savetxt

data_dir_demo  = "/content/drive/MyDrive/Colab Notebooks/Wiping_left_right/"

filename = data_dir_demo + "trajectory_data_wiping_left.npy"
filename_orig = data_dir_demo + "trajectory_data_wiping_left_orig.npy"

with open(filename, 'rb') as f:
  traj_standard = jnp.load(f) # nD x ntrajs (split) x nsamples x dim
  vel_standard = jnp.load(f)
  ts_new = jnp.load(f)
  scaler_all_t_combine = jnp.load(f) # 1/T (end time) nD x ntrajs (split) x 1
  minmax = jnp.load(f) # bounding cube (3 x 2)

with open(filename_orig, 'rb') as f:
  traj_all_combine_process = jnp.load(f) # nD x ntrajs (split) x nsamples x dim
  vel_all_combine_process = jnp.load(f)

# traj_load = traj_standard[:, :, :, :3]
traj_load = traj_all_combine_process[:, :, :, :3]

fig = go.Figure()
n_D = traj_load.shape[0]
split_indx = 0
for i in range(n_D):
  fig.add_trace(go.Scatter3d(x=traj_load[i, split_indx, :, 0], y=traj_load[i, split_indx, :, 1], z=traj_load[i, split_indx, :, 2], mode='lines',
                            line=dict(color='gray', width=4)))
  fig.add_trace(go.Scatter3d(x=traj_load[i, split_indx, :1, 0], y=traj_load[i, split_indx, :1, 1], z=traj_load[i, split_indx, :1, 2], mode='markers',
                            line=dict(color='red', width=1)))
  fig.add_trace(go.Scatter3d(x=traj_load[i, split_indx, -2:-1, 0], y=traj_load[i, split_indx, -2:-1, 1], z=traj_load[i, split_indx, -2:-1, 2], mode='markers',
                            line=dict(color='blue', width=1)))
fig.show()

nsamples = traj_load.shape[2]
# ts_new = np.linspace(0, 1, nsamples)

# Define the model

In [ ]:
class Func(eqx.Module):
    mlp: eqx.nn.MLP

    def __init__(self, data_size, width_size, depth, *, key, **kwargs):
        super().__init__(**kwargs)
        initializer = jnn.initializers.orthogonal()
        self.mlp = eqx.nn.MLP(
            in_size=data_size,
            out_size=data_size,
            width_size=width_size,
            depth=depth,
            activation=jnn.tanh,
            key=key,
        )
        model_key = key
        key_weights = jrandom.split(model_key, depth+1)

        for i in range(depth+1):
          where = lambda m: m.layers[i].weight
          shape = self.mlp.layers[i].weight.shape
          self.mlp = eqx.tree_at(where, self.mlp, replace=initializer(key_weights[i], shape, dtype = jnp.float32))

    @eqx.filter_jit
    def __call__(self, t, y, args):

        return self.mlp(y)

In [ ]:
## use the finite difference of the states also as a feature input to the vector field
class Funcd(eqx.Module):
    mlp: eqx.nn.MLP

    def __init__(self, data_size, width_size, depth, *, key, **kwargs):
        super().__init__(**kwargs)
        initializer = jnn.initializers.orthogonal()
        self.mlp = eqx.nn.MLP(
            in_size=2*data_size,
            out_size=2*data_size,
            width_size=width_size,
            depth=depth,
            activation=jnn.tanh,
            key=key,
        )
        model_key = key
        key_weights = jrandom.split(model_key, depth+1)

        for i in range(depth+1):
          where = lambda m: m.layers[i].weight
          shape = self.mlp.layers[i].weight.shape
          self.mlp = eqx.tree_at(where, self.mlp, replace=initializer(key_weights[i], shape, dtype = jnp.float32))

    @eqx.filter_jit
    def __call__(self, t, yd, args):

        return self.mlp(yd)
        # return self.mlp(jnp.concatenate([yd, jnp.array([t])]))

Here we wrap up the entire ODE solve into a model.

In [ ]:
class NeuralODE(eqx.Module):
    func: Func

    def __init__(self, data_size, width_size, depth, *, key, **kwargs):
        super().__init__(**kwargs)
        self.func = Func(data_size, width_size, depth, key=key)

    @eqx.filter_jit
    def __call__(self, ts, yd0):
        solution = diffrax.diffeqsolve(
            diffrax.ODETerm(self.func),
            diffrax.Tsit5(),
            t0=ts[0],
            t1=ts[-1],
            dt0=ts[1] - ts[0],
            y0=yd0,
            stepsize_controller=diffrax.PIDController(rtol=1e-3, atol=1e-6),
            saveat=diffrax.SaveAt(ts=ts),
        )
        return solution.ys

In [ ]:
## Predict the trajectory of finite difference of the states also
class NeuralODEd(eqx.Module):
    func: Funcd

    def __init__(self, data_size, width_size, depth, *, key, **kwargs):
        super().__init__(**kwargs)
        self.func = Funcd(data_size, width_size, depth, key=key)

    @eqx.filter_jit
    def __call__(self, ts, yd0):
        solution = diffrax.diffeqsolve(
            diffrax.ODETerm(self.func),
            diffrax.Tsit5(),
            t0=ts[0],
            t1=ts[-1],
            dt0=ts[1] - ts[0],
            y0=yd0,
            stepsize_controller=diffrax.PIDController(rtol=1e-3, atol=1e-6),
            saveat=diffrax.SaveAt(ts=ts),
        )
        return solution.ys

# Data loader

In [ ]:
def dataloader(arrays, batch_size, *, key):
    dataset_size = arrays[0].shape[0]
    assert all(array.shape[0] == dataset_size for array in arrays)
    indices = jnp.arange(dataset_size)
    while True:
        perm = jrandom.permutation(key, indices)
        (key,) = jrandom.split(key, 1)
        start = 0
        end = batch_size
        while end < dataset_size:
            batch_perm = perm[start:end]
            yield tuple(array[batch_perm] for array in arrays)
            start = end
            end = start + batch_size

# Train

In [ ]:
def main(
    dataset_size=1,
    batch_size=1,
    lr_strategy=(3e-3, 3e-3, 3e-3),
    steps_strategy=(5000, 5000, 5000),
    length_strategy=(0.3, 0.7, 1),
    width_size=64,
    depth=3,
    seed=1000,
    plot=True,
    print_every=100,
    save_every = 1000,
):
    key = jrandom.PRNGKey(seed)
    data_key, model_key, loader_key = jrandom.split(key, 3)
    file_name = "/content/drive/MyDrive/Colab Notebooks/Neural ODE/wiping_left.eqx"

    # ts, ys, ys_dot = get_data(dataset_size, key=data_key)

    ## training data

    # 2nd index is which trajectory of the split.

    split_i = 0
    ys = traj_load[:dataset_size, split_i]
    ts = ts_new

    _, length_size, data_size = ys.shape

    model = NeuralODE(data_size, width_size, depth, key=model_key)
    # Training loop like normal.
    #
    # Only thing to notice is that up until step 500 we train on only the first 10% of
    # each time series. This is a standard trick to avoid getting caught in a local
    # minimum.

    @eqx.filter_value_and_grad
    @eqx.filter_jit
    def grad_loss(model, ti, yi):
        y_pred = jax.vmap(model, in_axes=(None, 0))(ti, yi[:, 0])
        loss = jnp.mean((yi - y_pred) ** 2)
        return loss

    @eqx.filter_jit
    def make_step(ti, yi, model, opt_state):
        loss, grads = grad_loss(model, ti, yi)
        updates, opt_state = optim.update(grads, opt_state)
        model = eqx.apply_updates(model, updates)
        return loss, model, opt_state

    for lr, steps, length in zip(lr_strategy, steps_strategy, length_strategy):
        decay_scheduler = optax.cosine_decay_schedule(lr, decay_steps=steps, alpha=0.9)
        # decay_scheduler = optax.piecewise_constant_schedule(init_value=lr, boundaries_and_scales={int(steps*0.25):0.5, int(steps*0.5):0.5, int(steps*0.75):0.5})
        # decay_scheduler = optax.exponential_decay(init_value=lr, transition_steps=steps, decay_rate=0.98, transition_begin=int(steps*0.25), staircase=False)
        # decay_scheduler = optax.linear_schedule(init_value=lr, end_value=0.005*lr, transition_steps=steps, transition_begin=int(steps*0.25))
        optim = optax.adabelief(learning_rate=decay_scheduler)
        opt_state = optim.init(eqx.filter(model, eqx.is_inexact_array))
        _ts = ts[: int(length_size * length)]
        _ys = ys[:, : int(length_size * length)]
        # _ys_dot = []
        # ## No batches
        for step in range(steps):
          start = time.time()
          loss, model, opt_state = make_step(_ts, _ys, model, opt_state)
          end = time.time()
          if (step % print_every) == 0 or step == steps - 1:
            print(f"Step: {step}, Loss: {loss}, Computation time: {end - start}")
          if (step % save_every) == 0 or step == steps-1:
            eqx.tree_serialise_leaves(file_name, model)
        ## Batches
        # for step, (yi,) in zip(
        #     range(steps), dataloader((_ys,), batch_size, key=loader_key)
        # ):
        #     start = time.time()
        #     loss, model, opt_state = make_step(_ts, yi, model, opt_state)
        #     end = time.time()
        #     if (step % print_every) == 0 or step == steps - 1:
        #       print(f"Step: {step}, Loss: {loss}, Computation time: {end - start}")
        #     if (step % save_every) == 0 or step == steps-1:
        #       eqx.tree_serialise_leaves(file_name, model)

    # if plot:
    #     plt.plot(ts, ys[0, :, 0], c="dodgerblue", label="Real")
    #     plt.plot(ts, ys[0, :, 1], c="dodgerblue")
    #     model_y = model(ts, ys[0, 0])
    #     plt.plot(ts, model_y[:, 0], c="crimson", label="Model")
    #     plt.plot(ts, model_y[:, 1], c="crimson")
    #     plt.legend()
    #     plt.tight_layout()
    #     plt.savefig("neural_ode.png")
    #     plt.show()

    return ts, ys, model

In [ ]:
ts, ys, model = main()

Step: 0, Loss: 0.015772387385368347, Computation time: 7.094071865081787
Step: 100, Loss: 0.0005458690575323999, Computation time: 0.0063588619232177734
Step: 200, Loss: 0.0004894890007562935, Computation time: 0.0037429332733154297
Step: 300, Loss: 0.0004743468598462641, Computation time: 0.0035996437072753906
Step: 400, Loss: 0.00046222450328059494, Computation time: 0.00371551513671875
Step: 500, Loss: 0.0004328663635533303, Computation time: 0.0036230087280273438
Step: 600, Loss: 0.0003444298345129937, Computation time: 0.003644704818725586
Step: 700, Loss: 0.00013794272672384977, Computation time: 0.004027605056762695
Step: 800, Loss: 6.915207632118836e-05, Computation time: 0.00414276123046875
Step: 900, Loss: 4.821633410756476e-05, Computation time: 0.003934621810913086
Step: 1000, Loss: 3.877939161611721e-05, Computation time: 0.003679037094116211
Step: 1100, Loss: 3.198537524440326e-05, Computation time: 0.003984212875366211
Step: 1200, Loss: 3.8703728932887316e-05, Computatio

# Save the model

In [ ]:
file_name = "/content/drive/MyDrive/Colab Notebooks/Neural ODE/wiping_left.eqx"
eqx.tree_serialise_leaves(file_name, model)

# Load the model

In [ ]:
# load the saved model
file_name = "/content/drive/MyDrive/Colab Notebooks/Neural ODE/Wiping_franka_models/wiping_left.eqx"
dataset_size = 1
split_i = 0
ys = traj_load[:dataset_size, split_i, :, :3]
ts = ts_new
_, length_size, data_size = ys.shape
width_size=64
depth=3
seed=1000
key = jrandom.PRNGKey(seed)
data_key, model_key, loader_key = jrandom.split(key, 3)
model1 = NeuralODE(data_size, width_size, depth, key=model_key)
model_load= eqx.tree_deserialise_leaves(file_name, model1)

# Plot the predicted trajectories

In [ ]:
# Position trajectory
ys_act = jnp.copy(traj_load[:, 0, :])
fig = go.Figure()
# for ind_demo in range(n_D):
ind_demo = 0 # which demonstration for the given task.
ys_pred = model(ts, ys_act[ind_demo, 0]) # predicting only for the given demo
fig.add_trace(go.Scatter3d(x=ys_act[ind_demo, :, 0], y=ys_act[ind_demo, :, 1], z=ys_act[ind_demo, :, 2], mode='lines',
                          line=dict(color='gray', width=4)))
fig.add_trace(go.Scatter3d(x=ys_act[ind_demo, :1, 0], y=ys_act[ind_demo, :1, 1], z=ys_act[ind_demo, :1, 2], mode='markers',
                          line=dict(color='red', width=1)))
fig.add_trace(go.Scatter3d(x=ys_act[ind_demo, -2:-1, 0], y=ys_act[ind_demo, -2:-1, 1], z=ys_act[ind_demo, -2:-1, 2], mode='markers',
                          line=dict(color='blue', width=1)))

# fig.add_trace(go.Scatter3d(x=ys_pred_act_all[ind_demo][:, 0], y=ys_pred_act_all[ind_demo][:, 1], z=ys_pred_act_all[ind_demo][:, 2], mode='lines',
#                           line=dict(color='green', width=4)))

# fig.add_trace(go.Scatter3d(x=ys_pred_all[ind_demo][:, 0], y=ys_pred_all[ind_demo][:, 1], z=ys_pred_all[ind_demo][:, 2], mode='lines',
#                           line=dict(color='green', width=4)))

fig.add_trace(go.Scatter3d(x=ys_pred[:, 0], y=ys_pred[:, 1], z=ys_pred[:, 2], mode='lines',
                          line=dict(color='green', width=4)))

fig.show()
fig.write_html("position.html")

In [ ]:
# Velocity profile
ys_dot_pred = jax.vmap(model_load.func.mlp, in_axes=0)(ys_pred_all[ind_demo])*scaler_all_t_combine[ind_demo, split_i]
ys_dot_act =  vel_all_combine_process[1, split_i, :, :3][jnp.newaxis, :]
fig, axs = plt.subplots(1,3, figsize=(20,5))
for i in range(3):
  axs[i].plot(ys_dot_act[ind_demo, :, i], label='actual')
  axs[i].plot(ys_dot_pred[:,i]*(minmax[i, 1] - minmax[i, 0]), label='predicted')
plt.legend()